In [2]:
from tika import parser 
import os

doc_dir ='./data/input/'
def tika_parser(doc_dir):
    # pdf file paths
    pdf_files = [file for file in os.listdir(os.getcwd()) if file.endswith('.pdf')]

    # parsing pdf file 
    for pdf in pdf_files:
        parsed_pdf = parser.from_file(pdf)  
        pdf_data = parsed_pdf['content'].strip()
        with open(doc_dir+pdf.split('.')[0]+'.txt', 'w') as out_file:
            out_file.write(pdf_data)
                           
                           
tika_parser(doc_dir)

In [3]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.pipeline import ExtractiveQAPipeline

def get_pipe():

    document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
    dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
    document_store.write_documents(dicts)


    retriever = ElasticsearchRetriever(document_store=document_store)
    reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
    
    #reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)


    pipe = ExtractiveQAPipeline(reader, retriever)
    return pipe

pipe = get_pipe()

02/26/2021 13:22:30 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.315s]
02/26/2021 13:22:30 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.180s]
02/26/2021 13:22:30 - INFO - haystack.preprocessor.utils -   Converting data\input\APPARI_PRAVEEN KUMAR.txt
02/26/2021 13:22:30 - INFO - haystack.preprocessor.utils -   Converting data\input\Praveen_Kumar_Appari_cover_letter (1).txt
02/26/2021 13:22:30 - INFO - haystack.preprocessor.utils -   Converting data\input\Surisetty Resume ALBIS Plastics GmbH.txt
02/26/2021 13:22:30 - INFO - haystack.preprocessor.utils -   Converting data\input\VamsiKrishnaAppari_Resume (1).txt
02/26/2021 13:22:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.656s]
02/26/2021 13:22:31 - INFO - farm.utils -   Using device: CPU 
02/26/2021 13:22:31 - INFO - farm.utils -   Number of GPUs: 0
02/26/2021 13:22:31 - INFO - farm.utils -   Distributed Tra

In [4]:
def predict_answers(query):
    prediction = pipe.run(query= query, top_k_retriever=10, top_k_reader=5)

    print_answers(prediction, details="minimal")
    
predict_answers('who is from mannhein')

02/26/2021 13:22:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.232s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.17s/ Batches]

[   {   'answer': 'praveen1812kumar',
        'context': 'trasse 20, 1st Floor, Kirchheim, Heidelberg, 69142 \n'
                   'Germany Email: praveen1812kumar@gmail.com | Contact :+49 '
                   '17685607848 \n'
                   'Job Position – Regular Fullt'},
    {   'answer': 'praveen1812kumar',
        'context': 'n and \n'
                   'MD70 documentation according to ERP business needs. \n'
                   'mailto:praveen1812kumar@gmail.com\n'
                   'https://github.com/praveenkumarappari\n'
                   '• Gathering the re'},
    {   'answer': 'praveen1812kumar',
        'context': 'no: 109, Neckarau West, 68199 Mannheim| \n'
                   '+4917685607848 | E-Mail : praveen1812kumar@gmail.com \n'
                   'Git-Link: https://github.com/praveenkumarappari  \n'
                   '  Sex'},
    {   'answer': 'praveen1812kumar',
        'context': 'u with all the details. I am available via phone and '
                   'em